In [1]:
%run solution.ipynb

In [2]:
def calculate_delta_r1(T):
    return 2 * G0('Al', T) + 2 * G0('HCl', T) - 2 * G0('AlCl', T) - G0('H2', T)

def calculate_delta_r2(T):
    return G0('Al', T) + 2 * G0('HCl', T) - G0('AlCl2', T) - G0('H2', T)

def calculate_delta_r3(T):
    return 2 * G0('Al', T) + 6 * G0('HCl', T) - 2 * G0('AlCl3', T) - 3 * G0('H2', T)

In [3]:
Pa = 100000

def K1(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r1(T) / (R * T)) / Pa

def K2(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r2(T) / (R * T))

def K3(T, R = 8.314):
    return math.exp(-1 * calculate_delta_r3(T) / (R * T)) / Pa



In [4]:
T = 623.15
print(K1(T))
print(K2(T))
print(K3(T))

7.497131271125371e-07
7.221267004644182e-08
4.062199476332757e-49


In [5]:
print(D('AlCl', T))
print(D('AlCl2', T))
print(D('AlCl3', T))
print(D('HCl', T))
print(D('H2', T))

0.000191129931554728
0.00014747988098603198
0.00015245155020045148
0.0002736394738546893
0.0008762807050502889


In [6]:
PG = {
   'AlCl':0,
   'AlCl2':0,
   'AlCl3':0,
   'H2':0,
   'HCl':10000,
   'N2':90000
}

T = 623.15
HCl, AlCl, H2, AlCl2, AlCl3 = sy.symbols('HCl AlCl H2 AlCl2 AlCl3')
X = sy.Matrix([
    HCl**2 - K1(T) * AlCl**2 * H2,
    HCl**2 - K2(T) * AlCl2 * H2,
    HCl**6 - K3(T) * AlCl3**2 * H2**3,
    D('HCl', T) * (PG['HCl'] - HCl) + 2 * D('H2', T) * (PG['H2'] - H2),
    D('AlCl', T) * (PG['AlCl'] - AlCl) + 2 * D('AlCl2', T) * (PG['AlCl2'] - AlCl2) + \
        3 * D('AlCl3', T) * (PG['AlCl3'] - AlCl3) + D('HCl', T) * (PG['HCl'] - HCl)
])
Y = [AlCl, AlCl2, AlCl3, HCl, H2]
X.jacobian(Y)

Matrix([
[-1.49942625422507e-6*AlCl*H2,                       0,                                 0,                 2*HCl,         -7.49713127112537e-7*AlCl**2],
[                           0, -7.22126700464418e-8*H2,                                 0,                 2*HCl,           -7.22126700464418e-8*AlCl2],
[                           0,                       0, -8.12439895266551e-49*AlCl3*H2**3,              6*HCl**5, -1.21865984289983e-48*AlCl3**2*H2**2],
[                           0,                       0,                                 0, -0.000273639473854689,                 -0.00175256141010058],
[       -0.000191129931554728,   -0.000294959761972064,             -0.000457354650601354, -0.000273639473854689,                                    0]])

In [7]:
ans = newton_method(X, Y, [1, 1, 1, 1, 1])
P = {'AlCl': ans[0], 'AlCl2': ans[1], 'AlCl3': ans[2]}
print(P)

128
{'AlCl': 0.885717046967506, 'AlCl2': 8.14463673524594, 'AlCl3': 5977.45057480367}


In [14]:
def V(T, ro=2712):
    return (G('AlCl', T, PG, P) + G('AlCl2', T, PG, P) + G('AlCl3', T, PG, P)) * bank_td['Al']['mu'] * 1e9 / ro

In [15]:
print(G('AlCl', T, PG, P))
print(G('AlCl2', T, PG, P))
print(G('AlCl3', T, PG, P))
print(V(T))

-1.26883832187881e-7
-9.00299639930859e-7
-0.000683015277258636
-6805.49102293763
